In [1]:
import pandas as pd
import numpy as np
from numpy import random
random.seed(42)

#The following packages are not necessary to run the visible portions of this notebook

from collections import Counter, defaultdict
import biom
import h5py
from qiime2 import Artifact
from qiime2.plugins.feature_table.methods import rarefy
from skbio.stats import subsample_counts

# Filtering Metadata

In [2]:
ogfile = pd.read_csv('/mnt/zarrinpar/Pynchon/Notebooks/arrhythmia/agp/Metadata/alab4.csv', low_memory = False, sep='\t')
ogfile.shape

(98665, 2976)

In [3]:
#limit samples to those from the AGP study
ogfile = ogfile[ogfile.title =='American Gut Project']
ogfile.shape

(30075, 2976)

In [4]:
#Removed all non-stool samples,skin, tongue, etc.
ogfile = ogfile[ogfile.body_site == 'UBERON:feces']
ogfile.shape

(22657, 2976)

In [5]:
#remove animal samples
ogfile = ogfile[ogfile.host_common_name =='human']
ogfile.shape

(22647, 2976)

In [6]:
#extract month, day, year from date
ogfile['collection_timestamp'] = pd.to_datetime(ogfile['collection_timestamp'], format='mixed')
ogfile['hour'] = ogfile['collection_timestamp'].dt.hour
ogfile.loc[:, 'month'] = ogfile['collection_timestamp'].dt.month
ogfile.loc[:, 'day'] = ogfile['collection_timestamp'].dt.day
ogfile.loc[:, 'year'] = ogfile['collection_timestamp'].dt.year
ogfile.loc[:, 'day_num'] = ogfile['collection_timestamp'].dt.dayofyear.astype(float)
ogfile = ogfile.dropna(subset=['hour'])

In [7]:
#limit samples to AGP study and drop useless columns
stool_time_file = ogfile.dropna(axis=1, how='all')
stool_time_file.shape

(22640, 739)

## Filter juveniles

In [8]:
#consolidate age values from these two columns
na_mask = stool_time_file['age_corrected'] == 'Na'
stool_time_file.loc[na_mask, 'age_corrected'] = stool_time_file.loc[na_mask, 'age_years']

In [9]:
#remove hosts under 18
age_numeric = pd.to_numeric(stool_time_file['age_corrected'], errors='coerce')
adults = stool_time_file[(age_numeric >= 18) | age_numeric.isna()]
adults.shape

(21408, 739)

## Reformatting the strings referring to locations

In [10]:
adults.loc[:, 'state'] = adults['state'].astype(str)
adults.loc[:, 'geo_loc_name'] = adults['geo_loc_name'].astype(str)
adults.loc[:, 'country'] = adults['country'].astype(str)

adults.loc[:, 'geo_loc_name'] = adults['geo_loc_name'].str.replace(':Unspecified', '')
adults.loc[adults['state'] == 'USA:CO:Boulder', 'state'] = 'Boulder,Co,USA'

In [11]:
mask = adults.apply(lambda row: row['state'] in row['geo_loc_name'], axis=1)
adults.loc[mask, 'state'] = adults.loc[mask, 'geo_loc_name']
adults.loc[mask, 'geo_loc_name'] = 'Na'

mask = adults.apply(lambda row: row['country'] in row['geo_loc_name'], axis=1)
adults.loc[mask, 'country'] = adults.loc[mask, 'geo_loc_name']

In [12]:
un_colon = [state for state in adults.state.unique() if ":" in state]

for state in un_colon:
    mask = adults.state.apply(lambda x: x in state)
    adults.loc[mask, 'state'] = state

In [13]:
replacements = {
    'nan': 'Na',
    'WV': 'USA:WV',
    'Oberosterreich': 'Austria:Oberosterreich',
    'Vorarlberg': 'Austria:Vorarlberg',
    'MB': 'Canada:MB',
    'YT': 'Canada:YT',
    'Jiangsu': 'China:Jiangsu',
    'Auvergne Rhone-Alpes': 'France:Auvergne Rhone-Alpes',
    'RA': 'France:RA',
    'Upper Normandy': 'France:Upper Normandy',
    'Thuringia': 'Germany:Thuringia',
    'Emilia-Romagna': 'Italy:Emilia-Romagna',
    'Lazio': 'Italy:Lazio',
    'Lombardy': 'Italy:Lombardy',
    'Tuscany': 'Italy:Tuscany',
    "Hawke's Bay": "New Zealand:Hawke's Bay",
    'Otago': 'Norway:Otago',
    'Tirol': 'Austria:Tyrol',
    'Community of Madrid': 'Spain:Community of Madrid',
    'mazowieckie': 'Poland:mazowieckie',
    'Wellington': 'Norway:Wellington',
    'Akershus': 'Norway:Akershus',
    'Valencian Community': 'Spain:Valencian Community',
    'Skane lan': 'Sweden:Skane lan',
    'Uppsala lan': 'Sweden:Uppsala lan',
    'Vastra Gotaland County': 'Sweden:Vastra Gotaland County',
    'Aktiengesellschaft': 'Switzerland:Aktiengesellschaft',
    'USA:Arizona': 'USA:AZ'
}

adults.loc[:, 'state'] = adults['state'].replace(replacements)

In [14]:
adults.loc[:, 'latitude'] = adults.loc[:, 'latitude'].astype(str)
adults.loc[:, 'longitude'] = adults.loc[:, 'longitude'].astype(str)
adults.loc[:, 'coordinate'] = adults['latitude'] + ',' + adults['longitude']
adults.loc[:, 'city'] = adults['coordinate']

/tmp/ipykernel_90760/2196077375.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults.loc[:, 'coordinate'] = adults['latitude'] + ',' + adults['longitude']


In [15]:
city_mapping = {
    '30.4,-97.8': 'North Austin',
    '32.7,-117': 'San Diego',
    '32.8,-117': 'San Diego',
    '32.9,-117': 'San Diego',
    '32.7,-117.0': 'San Diego',
    '32.7,-117.1': 'San Diego',
    '32.7,-117.2': 'San Diego',
    '32.7,-117.3': 'San Diego',
    '32.8,-117.0': 'San Diego',
    '32.8,-117.1': 'San Diego',
    '32.8,-117.3': 'San Diego',
    '32.8,-117.2': 'San Diego',
    '32.9,-117.0': 'San Diego',
    '32.9,-117.1': 'San Diego',
    '32.9,-117.3': 'San Diego',
    '32.9,-117.2': 'San Diego',
    '33.0,-117.1': 'San Diego',
    '33.0,-117.3': 'San Diego',
    '33.0,-117.2': 'San Diego',
    '33,-117.2': 'San Diego',
    '33,-117.1': 'San Diego',
    '33,-117.3': 'San Diego',
    '34.75,-77.43': 'Jacksonville',
    '34.6,-77.3': 'Jacksonville',
    '34.718,-77.3': 'Jacksonville',
    '34.718,-77.4': 'Jacksonville',
    '34.718,-77.5': 'Jacksonville',
    '34.7,-77.3': 'Jacksonville',
    '34.7,-77.4': 'Jacksonville',
    '34.7,-77.5': 'Jacksonville',
    '34.75,-77.3': 'Jacksonville',
    '34.75,-77.4': 'Jacksonville',
    '34.75,-77.5': 'Jacksonville',
    '34.8,-77.3': 'Jacksonville',
    '34.8,-77.4': 'Jacksonville',
    '34.8,-77.5': 'Jacksonville',
    '37.8,-122.4': 'San Francisco',
    '37.9,-122.3': 'San Francisco',
    '37.4,-122.1': 'San Jose',
    '40.7,-74.0': 'New York City',
    '40.8,-74.0': 'New York City',
    '40.0,-105.2': 'Boulder',
    '40.027435,-105.251945': 'Boulder',
    '40.0,-105.3': 'Boulder',
    '40,-105.2': 'Boulder',
    '40,-105.3': 'Boulder',
    '41,-105.3': 'Boulder',
    '41,-105.2': 'Boulder',
    '42.3,-71.1': 'Boston',
    '42.4,-71.1': 'Boston',
    '51.5,-0.1': 'London',
    '51.6,-0.1': 'London',
    '51.5,-0.2': 'London',
    '51.3,-0.3': 'London',
    '51.4,-0.3': 'London',
    '51.5,-0.3': 'London',
    '51.6,-0.3': 'London',
    '51.7,-0.3': 'London',
    '51.3,-0.2': 'London',
    '51.4,-0.2': 'London',
    '51.5,-0.2': 'London',
    '51.6,-0.2': 'London',
    '51.7,-0.2': 'London',
    '51.3,-0.1': 'London',
    '51.4,-0.1': 'London',
    '51.5,-0.1': 'London',
    '51.6,-0.1': 'London',
    '51.7,-0.1': 'London',
    '51.3,0.0': 'London',
    '51.4,0.0': 'London',
    '51.5,0.0': 'London',
    '51.6,0.0': 'London',
    '51.7,0.0': 'London',
    '51.3,0': 'London',
    '51.4,0': 'London',
    '51.5,0': 'London',
    '51.6,0': 'London',
    '51.7,0': 'London',
    '51.3,0.1': 'London',
    '51.4,0.1': 'London',
    '51.5,0.1': 'London',
    '51.6,0.1': 'London',
    '51.7,0.1': 'London',
    '51.3,0.2': 'London',
    '51.4,0.2': 'London',
    '51.5,0.2': 'London',
    '51.6,0.2': 'London',
    '51.7,0.2': 'London',
    '40.5,-73.7': 'New York City',
    '40.5,-73.8': 'New York City',
    '40.5,-73.9': 'New York City',
    '40.5,-74.0': 'New York City',
    '40.5,-74': 'New York City',
    '40.5,-74.1': 'New York City',
    '40.5,-74.2': 'New York City',
    '40.6,-73.7': 'New York City',
    '40.6,-73.8': 'New York City',
    '40.6,-73.9': 'New York City',
    '40.6,-74.0': 'New York City',
    '40.6,-74': 'New York City',
    '40.6,-74.1': 'New York City',
    '40.6,-74.2': 'New York City',
    '40.7,-73.7': 'New York City',
    '40.7,-73.8': 'New York City',
    '40.7,-73.9': 'New York City',
    '40.7,-74.0': 'New York City',
    '40.7,-74': 'New York City',
    '40.8,-73.7': 'New York City',
    '40.8,-73.8': 'New York City',
    '40.8,-73.9': 'New York City',
    '40.8,-74.0': 'New York City',
    '40.8,-74': 'New York City',
    '40.9,-73.7': 'New York City',
    '40.9,-73.8': 'New York City',
    '40.9,-73.9': 'New York City',
    '40.9,-74.0': 'New York City',
    '40.9,-74': 'New York City',
    '42.2,-71.1': 'Boston',
    '42.2,-71.2': 'Boston',
    '42.3,-71': 'Boston',
    '42.3,-71.0': 'Boston',
    '42.3,-71.2': 'Boston',
    '42.4,-71': 'Boston',
    '42.4,-71.0': 'Boston',
    '42.4,-71.1': 'Boston',
    '42.4,-71.2': 'Boston'
}

adults.loc[:, 'city'] = adults.city.replace(city_mapping, regex=True)
adults.loc[adults.loc[:, 'state'] == 'Boulder,Co,USA', 'city'] = 'Boulder'

## Cleaning by BMI

In [17]:
clean_geo = adults 

In [18]:
# Consolidating categorical labels
values_to_replace = ['nan', 'None', 'Not provided', 'Missing: Not provided', 'unspecified', 'Unspecified', 'LabControl test', 'Not sure']
clean_geo = clean_geo.replace(values_to_replace, 'Na')
clean_geo = clean_geo.replace(['false','true'],['False','True'])

In [19]:
#create bmi categories
bins = [-float('inf'), 13, 18.5, 25, 30, 90, float('inf')]
labels = ['Na', 'Underweight', 'Normal', 'Overweight', 'Obese', 'Na']
clean_geo.loc[:, 'bmi'] = pd.to_numeric(clean_geo['bmi'], errors='coerce')
clean_geo.loc[:, 'bmi_cat_comb'] = pd.cut(clean_geo.loc[:, 'bmi'].astype(float), bins=bins, labels=labels, right=False, ordered = False)

## Cleaning by Date

In [21]:
#create season column
bins = [0, 80, 172, 264, 355,366]  
labels = ['Winter', 'Spring', 'Summer', 'Fall', 'Winter']

clean_geo.loc[:, 'season'] = pd.cut(clean_geo['day_num'], bins=bins, labels=labels, right=True, ordered = False)

In [22]:
#flip season for southern hemisphere countries
countries = ['Australia', 'Zimbabwe', 'New Zealand', 'Brazil', 'Paraguay', 'Argentina', 'Peru']
season_mapping = {'Summer': 'Winter', 'Winter': 'Summer', 'Fall': 'Spring', 'Spring': 'Fall'}
clean_geo.loc[clean_geo['country'].isin(countries), 'season'] = clean_geo.loc[clean_geo['country'].isin(countries), 'season'].replace(season_mapping)

In [24]:
#binning times
time_mapping = {
    '0.0': '1', '1.0': '1', '2.0': '3', '3.0': '3', '4.0': '5', '5.0': '5',
    '6.0': '7', '7.0': '7', '8.0': '9', '9.0': '9', '10.0': '11', '11.0': '11',
    '12.0': '13', '13.0': '13', '14.0': '15', '15.0': '15', '16.0': '17', '17.0': '17',
    '18.0': '19', '19.0': '19', '20.0': '21', '21.0': '21', '22.0': '23', '23.0': '23'
}

# Apply the mapping directly using replace
clean_geo.loc[:, 'hour'] = clean_geo['hour'].astype(str)
clean_geo.loc[:, 'col_bin'] = clean_geo['hour'].replace(time_mapping)

## Limit 1 sample per host

In [25]:
fully_cleaned = clean_geo
fully_cleaned.shape

(21408, 743)

In [26]:
n = 1
sampled_df = (
    fully_cleaned.groupby('host_subject_id')
    .apply(lambda x: x.sample(min(len(x), n)))
    .reset_index(drop=True)
)
eversamp = sampled_df.set_index('sample_id')

In [27]:
eversamp.shape

(18028, 742)